In [ ]:
import datetime as dt
import time
import matplotlib.pyplot as plt
import numpy as np

import pyxu.operator as pxop
import pyxu.opt.stop as pxos

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

### Generation of the source (using a single realization)

In [ ]:
#path = "simulated_data/my_simulations/simu0.csv"
path = "simulated_data/events3.csv"
beta = 5.0
hpL = HawkesLikelihood(path=path, beta=beta)
M = hpL.M

In [ ]:
hpL.plot_realization()

In [ ]:
start = time.time()
lip = hpL.opA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)

In [ ]:
print("Average number of arrivals per process:", np.mean(hpL.k))

### Generation of the source (using multiple realizations)

In [ ]:
totalProcesses = 15
beta = 5.0

paths = [f"simulated_data/events{i}.csv" for i in range(totalProcesses)]
allHPs = np.full(totalProcesses, None)

for i in range(totalProcesses):
    print(f"Initializing process {i+1}...")
    allHPs[i] = HawkesLikelihood(path=paths[i], beta=beta)
M = allHPs[0].M

In [ ]:
# Define big forward operator A: R^{M + M^2} -> R^{K1 + ... + Kn + totalProcesses*M}
bigA = pxop.vstack([allHPs[i].opA for i in range(totalProcesses)])

# Define big E operator E1(x_1, ..., x_K1) + E2(x_{K1+1}, ..., x_{K1+K2}) + ...
# E: R^{K1 + ... + Kn} -> R
bigE = pxop.hstack([allHPs[i].E for i in range(totalProcesses)])

# Define big -log(likelihood operator)
bigNegLogL = bigE * bigA

In [ ]:
start = time.time()
lip = bigA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)

### Defining solvers

In [ ]:
## Parameters of the solvers
# Base
remove = True  # originally True
min_iterations = 1
# Stop
eps = 9e-4  # relative error for objective function
tmax = 300.0  # seconds
eps_dcv = 1e-2
# PFW
ms_threshold = 0.8  # original value 0.8, the lower the more atoms will HP select
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

# Parameter for reconstruction
lambda_ = 18e1

In [ ]:
pfw = pyfwl.PFWLasso(
    forwardOp=hpL.opA,
    convexOp=hpL.E,
    lambda_=lambda_,
    M=hpL.M,
    negLogL=hpL.negLogL,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=True,
)
# pfw = pyfwl.PFWLasso(
#     forwardOp=bigA,
#     convexOp=bigE,
#     lambda_=lambda_,
#     M=M,
#     negLogL=bigNegLogL,
#     ms_threshold=ms_threshold,
#     init_correction_prec=init_correction,
#     final_correction_prec=final_correction,
#     remove_positions=remove,
#     min_correction_steps=correction_steps,
#     show_progress=True,
# )

In [ ]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [ ]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


### Evaluation of the solve

In [ ]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print("Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

In [ ]:
# Evaluate sparsity
def supp(arr):
    return np.where(np.abs(arr) > 1e-3)[0]
print("Percentage of nonzero coefficients:", len(supp(data_p["x"]))/len(data_p["x"]) * 100)

# Arrange intensities in a M x M matrix
intensities = np.zeros((M, M))
mus = np.zeros((M,1))
for i in range(M):
    intensities[i, :] = data_p["x"][(i*(M + 1) + 1):(i*(M + 1) + M + 1)]
    mus[i] = data_p["x"][i*(M+1)]

# for i in range(M):
#     print(f"Process {i}:", data_p["x"][(i*(M + 1)):(i*(M + 1) + M + 1)])

In [ ]:
# Visualize solution
fig, ax = plt.subplots(1, 2, figsize=(8,5))

# Plot alpha's
im = ax[1].imshow(intensities/(beta*beta), # la norme L1 est alpha/beta^2
                cmap='viridis', 
                interpolation='nearest', 
                extent=[1, M, M, 1])
fig.colorbar(im, ax=ax[1])  # Add a colorbar to show the scale
ax[1].set_title('$||h_{ij}||_1$ for $i,j=1,...,M$')

# Plot mu's
im = ax[0].imshow(mus,
                  cmap='viridis',
                  interpolation='nearest',
                  extent=[1, 2, M, 1])
fig.colorbar(im, ax=ax[0])  # Add a colorbar to show the scale
ax[0].set_title('$\mu_i$ for $i=1,...,M$')

plt.show()

# for i in range(M):
#     print(f"mu{i+1}:", mus[i])

# print(intensities/(beta*beta))

In [ ]:
# Plot objective function value
mini = hist_p["Memorize[objective_func]"][-1]

plt.figure(figsize=(10, 8))
plt.suptitle("Objective function values")
plt.subplot(211)
plt.title("Log-scale")
plt.yscale("log")
plt.plot(
    hist_p["duration"],
    (hist_p["Memorize[objective_func]"] - mini),
    label="PFW",
    marker="+",
)

plt.legend()
plt.ylabel("OFV")

plt.subplot(212)
plt.plot(
    hist_p["duration"],
    (hist_p["Memorize[objective_func]"] - mini),
    label="PFW",
    marker="+",
)

plt.legend()
plt.ylabel("OFV")
plt.xlabel("Solving time [s]")
plt.title("Linear scale")
plt.show()

In [ ]:
# Plot DCV values
plt.figure(figsize=(10, 5))
plt.title("Dual certificate values")
plt.plot(hist_p["duration"], hist_p["AbsError[dcv]"], label="PFW", marker="+")
plt.xlabel("Solving time [s]")
plt.legend()
plt.show()